# Hands-on experiment: k-means Clustering by Semidefinite Programming

Clustering is an unsupervised machine learning problem in which we try to partition a given data set into $k$ subsets based on distance between data points or similarity among them. The goal is to find $k$ centers and to assign each data point to one of the centers such that the sum of the square distances between them are minimal [1]. This problem is known to be NP-hard. 

##  Problem Definition
---
### 1. Clustering problem
Given a set of $n$ points in a $d-$dimensional Euclidean space, denoted by
\begin{equation*}
S = \{ \mathbf{s}_i = (s_{i1}, \cdots, s_{id})^\top~\in \mathbb{R}^d ~~ i = 1, \cdots, n\}
\end{equation*}
find an assignment of the $n$ points into $k$ disjoint clusters $\mathcal{S} = (S_1, \cdots, S_k)$ whose centers are $\mathbf{c}_j(j = 1, \cdots, k)$ based on the total sum of squared Euclidean distances from each point $\mathbf{s}_i$ to its assigned cluster centroid $\mathbf{c}_i$, i.e.,
\begin{equation}
f(S,\mathcal{S}) = \sum_{j=1}^k\sum_{i=1}^{|S_j|}\|\mathbf{s}_i^{j} - \mathbf{c}_j \|^2,\nonumber
\end{equation}
where $|S_j|$ is the number of points in $S_j$, and $\mathbf{s}_i^{j} $ is the $i^{th}$ point in $S_j$.


## EXERCISE 1 - SDP relaxation and conditional gradient method
--- 

### 1. Relaxation of the problem
[1] proposes an SDP-relaxation to approximately solve the abovementioned model-free $k-$means clustering problem.
The resulting optimization problem (See section (2) of [1] for details of this relaxation) takes the standard semidefinite programming form 

\begin{equation}
\mathbf{X^\star} \in \arg \min_{ \mathbf{X} } \bigg\{ \text{Tr}(\mathbf{CX}) : ~\underbrace{\mathbf{X} \mathbf{1} =  \mathbf{1}}_{A_1(\mathbf{X}) = b}, ~\underbrace{\mathbf{X}^\top \mathbf{1} =  \mathbf{1}}_{A_2(\mathbf{X}) = b},~\underbrace{\mathbf{X}\geq 0}_{\mathbf{X} \in \mathcal{K}},~\underbrace{\mathrm{Tr}(\mathbf{X}) \leq \kappa, ~\mathbf{X} \in \mathbb{R}^{p\times p} ,~\mathbf{X}\succeq 0 }_{\mathcal{X}} \bigg\}, \label{eq:cluster_sdp}\tag{1}
\end{equation}
where $C \in \mathbb{R}^{p \times p}$ is the Euclidean distance matrix between the data points. 
$\mathrm{Tr}( \mathbf{X}) \leq \kappa$ enforces approximately low-rank solutions, the linear inclusion constraint $\mathbf{X} \geq 0 $ is element-wise nonnegativity of ${\mathbf{X}}$, the linear equality constraints $ \mathbf{X}\mathbf{1} = \mathbf{1}$ and $ \mathbf{X}^{\top} \mathbf{1} = \mathbf{1}$ require row and column sums of $ \mathbf{X}$ to be equal to 1's, and $ \mathbf{X} \succeq 0$ means that $ \mathbf{X}$ is positive semi-definite. 

Recall that $\mathrm{Tr}( \mathbf{X}) = \| \mathbf{X}\|_\ast$ for any positive semi-definite matrix $ \mathbf{X}$.


### 2. Algorithm
---

Excluding $\mathcal{X}$, Program \eqref{eq:cluster_sdp} can be reformulated as

\begin{equation}
\begin{aligned}
&\min_{\mathbf{X} \in \mathcal{X}}  f(\mathbf{X}) + g(A(\mathbf{X})) \\
&\text{subject to }  B(\mathbf{X}) \in \mathcal{K}
\end{aligned}\label{eq:hcg}\tag{2}
\end{equation}
where $f(x) = \text{Tr}(\mathbf{CX}) $  is a smooth convex function,  $g$ is the indicator function of the singleton $\{\mathbf{b}\}$ and ${\mathcal{K}}$ is the positive orthant (i.e. the set $\mathcal{K} = \{\mathbf{X} \in \mathbb{R}^{p\times p}: \mathbf{X} \geq 0\}$) for which computing the projection is easy. This means that we can do simple gradient updates on this part of the problem. To account for the constraint set $\mathcal{x}$, we can rely on the powerful linear minimization oracle to tackle the fact that $\mathbf{X} \in \mathcal{X}$ without requiring projections. Recall that the linear minimization oracle (LMO) is defined as follows
 \begin{equation*}
\text{lmo}_{\mathcal{X}}(\mathbf{Z})  = \arg \min_{\mathbf{X} \in \mathcal{X}} \langle \mathbf{X},\mathbf{Z} \rangle \qquad \text{where}\qquad \langle \mathbf{X},\mathbf{Z} \rangle = \text{Tr}(\mathbf{Z}^\top\mathbf{X}).
\end{equation*}
One of the main advantages of leveraging LMOs is to avoid doing costly projection operations, which explains the increasing popularity of this method (e.g. [2,5]). Indeed, in this setting, projecting onto $\mathcal{X}$ would require computing a full singular value decomposition (SVD), a very expensive operation.


Note that the classical Frank-Wolfe method does not apply to this problem due to nonsmooth term $g$. 
In the sequel, we will attempt to solve this problem with the HomotopyCGM method proposed in [2] to handle the non-smooth problems with a conditional gradient based method. This approach reformulates the problem using smoothing and homotopy, which in term leads to the gradient expression found at the second line of the algorithm below. The details of the transformation can be found in [2]. The full Homotopy CGM algorithm to solve  \eqref{eq:hcg} is stated just below.


---
**Conditional Gradient Method(CGM) for composite problems**

---
$\textbf{1.}~ \text{Choose } \mathbf{X}_0 \in\mathcal{X}$ and $\beta_0 >  0$ 

$\textbf{2.}~\text{For } k=1, 2, \ldots$  perform: 
\begin{equation*}
\begin{cases}
\gamma_k &= 2/(k+1), ~\text{and}~ \beta_k = \beta_0 / \sqrt{k+1} \\
\mathbf{V}_k &= \beta_k \nabla f(\mathbf{X}_k) + A^\top (A\mathbf{X}_k - \mathbf{b}) + (\mathbf{X}_k - \text{proj}_{\mathcal{K}}(\mathbf{X}_k))\\
\hat{\mathbf{X}}_k &:= \mathrm{argmin}_{x \in \mathcal{X}} \left \langle  \mathbf{V}_k,  \mathbf{X} \right \rangle \\
\mathbf{X}_{k+1} &:= (1-\gamma_k)\mathbf{X}_k + \gamma_k\hat{\mathbf{X}}_k
\end{cases}
\end{equation*} 
$\textbf{3.}~ \text{Output:} ~ \mathbf{X}_k$ 

---

### 3. Dataset:
    
We use a similar setup described by [3]. We use the fashion-MNIST data in [4] which is released as a possible replacement for the MNIST handwritten digits. Each data point is a 28x28 grayscale image, associated with a label from 10 classes. The figure below depicts 3 row samples from each class. Classes are labeled from 0 to 9. 

First, we extract the meaningful features from this dataset using a simple 2 layers neural network with a sigmoid activation. Then, we apply neural network to 1000 test samples from the same dataset, which gives us a vector $\mu \in \mathbb{R}^{10}$ where each entry represents the probability being in that class. 
Then, we form the pairwise distance matrix $\boldsymbol{C}$ by using this probability vectors.

**N.B.** In the code, you do not need to worry about any of the processing details mentioned here. You are directly given the matrix $\mathbf{C}$.


![Fashion-MNIST example](fig/fashion-mnist-sprite.png)


### Usual imports first

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse.linalg import eigsh, svds, eigs
from math import sqrt
import scipy.io as sio
import random
import numpy.matlib
import time
import matplotlib.pyplot as plt
from helpers import *

### Load data

In [ ]:
Problem = sio.loadmat('data/clustering_data.mat')
C = np.double(Problem['C']) # euclidean distance matrix
N = np.int(Problem['N']) # number of data points
k = Problem['k'] # number of clusters
opt_val = Problem['opt_val'] # optimum value 
b = np.double(np.ones(N)) # vector of ones, for the constraint

### 4. Questions

---
#### <span style="font-variant:small-caps;">Question 1 *(Pen & paper)*</span>
Show that the domain $\mathcal{X} = \{\mathbf{X}: \mathrm{Tr}(\mathbf{X}) \leq \kappa,~\mathbf{X} \in \mathbb{C}^{p\times p} ,~\mathbf{X} \succeq 0\}$ is a convex set. 

<span style="font-variant:small-caps;">Recall:</span> $\mathcal{X}$ is a convex set if and only if for any $\mathbf{X},\mathbf{Y} \in \mathcal{X}$ and $\alpha \in [0,1]$, $$\alpha\mathbf{X} + (1-\alpha)\mathbf{Y} \in \mathcal{X}$$



---
#### <span style="font-variant:small-caps;">Question 2 *(Pen & paper)*</span>
Given a linear inclusion constraint $T\mathbf{X} \in \mathcal{Y}$, the corresponding quadratic penalty function is given by
$$
\text{QP}_\mathcal{Y}(x) = \text{dist}^2(T\mathbf{X}, \mathcal{Y}) = \min_{y \in \mathcal{Y}} \| y-T\mathbf{X}\|^2.
$$
Write down the constraints in equation (1) in the quadratic penalty form and **show that** the penalized objective takes the form
$$
f(\mathbf{X}) + \frac{1}{2\beta} \|A_1(\mathbf{X}) - b_1\|^2 + \frac{1}{2\beta} \|A_2(\mathbf{X}) - b_2\|^2 + \frac{1}{2\beta}\text{dist}^2(T\mathbf{X}, \mathcal{K}), 
$$
and **show that** the gradient of the penalized objective is equal to ${v_k/}{\beta}$ in the algorithm.

<span style="font-variant:small-caps;"> Hints: </span> 
1. You can write $\text{dist}^2(T\mathbf{X}, \mathcal{Y}) = \| y^\ast - T\mathbf{X} \|^2$, where $y^\ast = \arg \min _{y\in \mathcal{Y}} \| y - T\mathbf{X} \|^2$, and take the derivative with respect to $\mathbf{X}$ without worrying about $y^\ast$ depending on $\mathbf{X}$, thanks to [Danskin's theorem](https://en.wikipedia.org/wiki/Danskin%27s_theorem).  
2. $\langle C,X \rangle = \mathrm{Tr}(\mathbf{C}^\top \mathbf{X})$ and $\frac{\partial \mathrm{Tr}(\mathbf{C}^\top \mathbf{X})}{\partial \mathbf{X}} = \mathbf{C}^\top $
3. Note that $\frac{\partial \mathbf{a}^\top \mathbf{X} \mathbf{b}}{\partial \mathbf{X}} = \mathbf{ab}^\top$ and  $\frac{\partial \mathbf{b}^\top \mathbf{X} \mathbf{X}^\top c}{\partial\mathbf{X}} = \mathbf{X}(\mathbf{bc}^\top + \mathbf{cb}^\top)$

---
#### <span style="font-variant:small-caps;">Question 3 *(Coding)*</span>

Complete the missing lines in the function definition of `HomotopyCGM`, and run the code after it to solve the $k$-means clustering problem. Visualize the results and SDP solution $\mathbf{X}$ after $5000$ iterations using the provided function `plotCGM`. What is the final objective value? Is it below the optimal value provided to you?

(<span style="font-variant:small-caps;"> Hint: </span> Note that when $\mathcal{X}$ is as given in equation (2), $\kappa u u^\top \in \text{lmo}_{\mathcal{X}}(X)$, where $u$ is the eigenvector corresponding to the smallest eigenvalue of $X$.)

---

### Define operators
We provide 2 operators:
1. `A1`: Linear operator that takes the row sums
2. `At2`: Conjugate of operator A1
3. `A2`: Linear operator that takes the column sums 
4. `At2`: Conjugate of operator A2

In [ ]:
A1 = lambda x: np.sum(x, axis = 1)
At1 = lambda y: np.transpose(np.matlib.repmat(y, N, 1))
A2 = lambda x: np.sum(x, axis = 0)
At2 = lambda y: (np.matlib.repmat(y, N, 1))

b = np.double(np.ones(N))

### The algorithm

In [ ]:
def HomotopyCGM(kappa=10, maxit=np.int(1e3), beta0=1):
    # Initialize
    X = np.zeros((N,N))
    AX_b = 0.0
    output = dict(); 
    output['feasibility1'] = [] # norm(A1(X)-b1)/norm(b1)
    output['feasibility2'] = []  # dist(X, \mathcal{K})
    output['objective'] = [] # f(x)
    output['cur_iter'] = []
    output['cur_time'] = []

    #u = np.zeros((N,1))
    iter_track = np.unique(np.ceil(np.power(2, np.linspace(0,20,50))))
    start = time.time()
    for iteration in range(1, maxit+1):
        
        # Update Step Size
        gamma = ???
        
        # Update beta
        beta_ = ???
        
        # Write down the vk to use in the lmo (eigenvalue routine)
        vk = beta_*C + At1(A1(X) - b) + At2(A2(X) - b) + 1e3*np.minimum(X, 0) 
        vk = 0.5*(vk + vk.T)
        
        # Linear minimization oracle
        # use eigsh function with proper settings to calculate the lmo
        # See Reference: https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.eigsh.html#scipy.sparse.linalg.eigsh 
            q, u = eigsh(???, tol=1e-16, which='SA')
        u = sqrt(kappa)*u
        X_bar = np.outer(u,u)
        
        # Obtain A*Xbar - b
        AX_bar_b = A1(X_bar)-b
        
        # Update A*X - b
        AX_b = (1.0-gamma)*AX_b + gamma*(AX_bar_b)
        
        # Update X
        X = ???
                
        if any(iteration == iter_track) or iteration==maxit:
            output['feasibility1'].append(np.linalg.norm(AX_b)/N)
            output['feasibility2'].append(np.linalg.norm(np.minimum(X,0), ord='fro'))
            output['objective'].append(np.sum(C.flatten()*X.flatten()))
            output['cur_iter'].append(iteration)
            output['cur_time'].append(time.time()-start)
            print('{:03d} | {:.4e}| {:.4e}| {:.4e}|'.format(iteration, output['feasibility1'][-1], output['feasibility2'][-1],output['objective'][-1]))
            
    return X, outputa

### Run the algorithm

In [ ]:
X, output_CGM = HomotopyCGM(10, np.int(5e3), 1)

### Visualize the results

In [ ]:
plot_result(output_CGM, X)

### Get the assignments from the result of the SDP
Getting the assignments requires going back to the $10$ dimensional space discussed before, and using the coordinates multiplied with the obtained matrix to construct a "denoised" version of the data points. This allows then to find the clusters from these $10$ dimensional data. See [3] for more details. Our implementation is the python reimplementation of their matlab code which can be found on [github](https://github.com/solevillar/kmeans_sdp).

In [ ]:
center_CGM,assign_CGM = sdp_rounding(X,10)

In [ ]:
print('Misclassification rate for CGM: {:.4f}'.format(misclassification_rate(assign_CGM)))

### Visualize samples and predicted labels

In [ ]:
vis_samples(assign_CGM)

## EXERCISE 2 - SDP relaxation and inexact Augmented Lagragian 
---

### 1. Burer-Monteiro Factorization of Problem (1)

Following the approach of [6], we will now turn to a more scalable approach, in which we reformulate Problem (1) using a factorized formulation for $\mathbf{X}=\mathbf{VV}^\top$, where $\mathbf{V}\in\mathbb{R}^{p\times r}$.

\begin{equation}
\mathbf{V^\star} \in \arg \min_{ \mathbf{V} \in \mathbb{R}^{p\times r} } \bigg\{ \text{Tr}(\mathbf{CVV}^{\top}) : ~\underbrace{\mathbf{VV}^{\top} \mathbf{1} =  \mathbf{1}}_{A(\mathbf{VV}^{\top}) = b},~\underbrace{\|\mathbf{V}\|_F^2\leq \kappa ,~\mathbf{V}\geq 0 }_{\mathcal{X}} \bigg\}, \tag{3}\label{3}
\end{equation}

However, while we gain in storage for the decision variable, the problem becomes nonconvex as a consequence. We replace the condition of $\mathbf{X}$ being positive semidefinite and having nonnegative entries by the much stronger yet easier to enforce contraint $\mathbf{V}\geq 0$. The reasoning behind this relaxation is discussed in [7]. 

*N.B.1* Note that this relaxation circuments the expensive projection due to a trace constraint, as projecting on a Froebenius-ball of radius $\kappa$ is easy. This means that the problem can be solved without resorting to a conditional gradient method.

*N.B.2* $\|\mathbf{V}\|_F^2 = \mathrm{Tr}(\mathbf{VV}^{\top}) =\mathrm{Tr}(\mathbf{X})$ which is the constraint we previously had in (1).


However, the nonconvex formulation of \eqref{3} suggests turning to an augmented Lagrangian approach, and defining the dual variable $\mathbf{y} = [\mathbf{y}_1, \mathbf{y}_2]$, where $\mathbf{y}_{1,2} \in \mathbb{R}^{p}$, we obtain the following expression for the augmented Lagrangian formulation

\begin{equation}
L_\beta(\mathbf{V},\mathbf{y}) = \langle C, \mathbf{VV}^\top\rangle + \frac{\beta}{2}\|A_1(\mathbf{VV}^\top) - \mathbf{b}_1\|^2 + \frac{\beta}{2}\|A_2(\mathbf{VV}^\top) - \mathbf{b}_2\|^2 + \langle A_1(\mathbf{VV}^\top)-\mathbf{b}_1, \mathbf{y}_1\rangle + \langle A_2(\mathbf{VV}^\top)-\mathbf{b}_2, \mathbf{y}_2\rangle \tag{4}\label{eq:al}
\end{equation}

Overall, the problem that needs to be solved now is 
\begin{equation}
\min_\mathbf{V}\max_\mathbf{y}\mathcal{L}_\beta(\mathbf{V},\mathbf{y}) + g(\mathbf{V})
\end{equation}

which leads to the following algorithm

---
**Inexact Augmented Lagranfian Method (iALM) for solving**

---

$\textbf{1.}~$Initialize the primal and dual variables $\mathbf{V}_0 \in \mathbb{R}^{p\times r}, \mathbf{y}_0 \in \mathbb{R}^m, \text{step size } \sigma_0 > 0$

$\textbf{2.}~$**For** $k=1, 2, \ldots$  perform: 

$\textbf{3.}~~~~~~~$**(Update tolerance)**$~\beta_k = \beta_{k-1}\omega^{1/4},\quad m^{inner}_k = m^{inner}_{k-1}\omega%, \quad \epsilon_{k+1}=1/\beta_k$

$\textbf{4.}~~~~~~~$**(Inexact primal solution)** Run  $m^{inner}_k$ iterations of  Accelerated Proximal Gradient Descent (APGM) to obtain the inexact solution $\mathbf{V}_{k+1}$ of

$$
\min_{\mathbf{V} \in \mathcal{X} }\mathcal{L}_\beta(\mathbf{V},\mathbf{y}_k) + g(\mathbf{V})
$$

$\textbf{5.}~~~~~~~$**(Update dual step size)**
$\sigma_{k+1} = \sigma_1 \min\left(\frac{\|A(\mathbf{V}_1\mathbf{V}_1^\top)\| \log^2 2}{\|A(\mathbf{V}_{k+1}\mathbf{V}_{k+1}^\top)\|(k+1)\log^2(k+2)},\frac{1}{k}\right)$


$\textbf{6.}~~~~~~~$**(Dual ascent)** $\mathbf{y}_{k+1}=\mathbf{y}_k + \sigma_{k+1}A(\mathbf{V}_{k+1})$

$\textbf{8.}~$**return** $\mathbf{V}_{k+1}$ as an (approximate) stationary point.

---


#### Remarks
1. Increasing the number of iterations for the subsolver (APGM) used in step $4$ of the algorithm gradually increases the accuracy of the subproblems. 
2. The special update rule for dual step size in step $5.$ of the algorithm ensures the boundedness of the dual.
3. The Lagrangian parameter $\beta$ is updated linearly at each outer iteration to ensure the gradual enforcements of the constraints. 


### 3. Questions

The point of this exercise will be to implement the function formulation and derive the corresponding gradient, in order to be able to compare the performances with the HomotopyCGM method. We won't ask you to implement everything from scratch

---
#### <span style="font-variant:small-caps;">Question 1 *(Pen & paper)*</span>

Derive the gradient of the augmented Lagrangian in \eqref{eq:al} with respect to $\mathbf{V}$. This involves again taking the derivative with respect to a matrix. Reuse the hints of question $2.$ of the previous parts as well as the following hint.

<span style="font-variant:small-caps;"> Hint: </span> For a symmetric convex function $f$, i.e.  $f(\mathbf{X}) = f(\mathbf{X}^\top)$, let $\mathbf{X} = \mathbf{VV}^\top$, then $\nabla_\mathbf{V}f(\mathbf{VV}^\top) = 2\nabla_\mathbf{X}f(\mathbf{X})\mathbf{V}$



---

#### <span style="font-variant:small-caps;">Question 2 *(Coding)*</span> 

Complete the missing lines in the function definition of `apgm_clustering` and `clustering_proj`. 

- In `apgm_clustering`, you have to fill in f, the augmented Lagrangian, while its gradient with respect to $\mathbf{V}$ is given to you.

- In `clustering_proj`, you have to write down the projection onto intersection of two sets which are the positive orthant and the $L_2$ norm-ball of radius $\sqrt{k}$.

Run the code after it to solve the $k$-means clustering problem. Visualize the results and SDP solution $\mathbf{X}$ after $2\cdot10^4$ function evaluations using the provided function `apgm_al_clustering`. What is the final objective value? Is it below the optimal value provided to you? How does it perform against HCGM?

**N.B.** In order to have an efficient implementation,  it is important to use efficient numpy functions. As we're working with numpy arrays, it is better to use `np.sum()` than `sum()`. Also, the function `.dot( )` should be used for matrix-matrix and matrix-vector multiplications.

#### 0. Define the parameters for the algoritm

In [ ]:
n = N/k
m = k
r = 20;

vec = lambda x: np.reshape( x, [np.size(x), 1]);
normC = np.linalg.norm(C,2)

Aef =  lambda U : U.dot(sum(U, 0)) 
# Since UU^T is symmetric, you only need one function here
# Summing row-wise or column-wise will yield the same result

f_u = lambda u: sum(vec(u * (C.dot(u))));
proj = lambda u: clustering_proj(u, k);
get_feas =  lambda u:  np.concatenate([Aef(u) - b,Aef(u) - b])
gamma_rule = lambda beta_,yc: .5*beta_/(2*N + beta_*normC + sqrt(2*N)*np.linalg.norm(beta_*yc-1,2));
myFunc = lambda u, yc, beta_: apgm_clustering(u, C, yc, b, b, beta_, r, N,nargout=2);


# Defining the initial parameters for the experiment

params = dict()
params['iters'] = int(1e4);#Number of iterations - RUN FOR 1.5e4 to get definitive results
params['verbose'] = 1;
params['Uinit'] = np.random.randn(N, r) # Random initialization
params['ycinit'] = np.zeros([2*b.shape[0]])
params['gamma'] = 1e5;
params['beta0'] = 10; # initial value for the penalty parameter
params['sigma0'] = 10;# initial value for dual step size

#### 1. `apgm_clustering` is the function containing the function and gradient evaluation.
Fill in `f` and `g`.

In [ ]:
def apgm_clustering(U, C, yc, b, b2, beta_, r, N, nargout=1):
    # This function evaluates the augmented Lagrangian and optionally the gradient of it if nargout=2
    # It is written in this form to allow for compact and efficient function evaluations.
    #     f here refers to L(u, beta, yc) = f(u) + beta/2*||A(u)-b||**2 + <yc, A(u) - b>
    #     g here refers to grad_u(L)
    # INPUT:   U: the factorized matrix element
    #          C: the pairwise distances matrix
    #          yc: the dual variable
    #          b: the vector of the first row-wise constraint
    #          b2: the vector of column-wise constraint
    #          beta_: the augmented Lagrangian parameter
    #          r: the rank of the factorized matrix = U.shape()[1]
    #          N: the number of data = U.shape()[0]
    #          nargout = 1
    #  OUTPUT: f: the augmented Lagrangian evaluated at U
    #          g: the gradient of the augmented Lagrangian with respect to U.
    vec = lambda x: np.reshape( x, [np.size(x), 1])
    f = ???
    
    if nargout > 1:
        g = C.dot(U) + (beta_*( np.tile((Aef(U)-b)[np.newaxis].T,[1,r])*(np.sum(U,0)) 
                     + np.tile(((Aef(U)-b)[np.newaxis]).dot(U),[N,1]))
                     + np.tile(yc[0:len(b),np.newaxis] , [1, r])*np.sum(U,0)
                     + np.tile(yc[len(b):2*len(b),np.newaxis].T.dot(U), [N,1]))
        return f , g
    return f

#### 2. `clustering_proj` is the function containing the projection on the intersection of  the positive orthant and the L2 norm-ball of radius sqrt{k}.
Fill in the projection operation.

In [ ]:
def clustering_proj(u, k):
    # This function takes the projection onto intersection of two sets which
    # are non-negativity and l2 norm-ball of radius sqrt(k)
    # u > 0 and ||u|| <= sqrt(k)
    u_ = ??? # Projection on positive orthant
    u_ = ??? # Projection on l2 norm-ball of radius sqrt(k)

    return u_

#### 3. `clustering_proj` is the main function
There's nothing to fill in it, but it's left here to show the flow of the optimization.

In [ ]:
def apgm_al_clustering(myFunc, proj, gamma_rule, get_feas, params):
    # APGM solver, inner problems are solved inexactly with a predefined  and then dual update.
    # Augmented Lagrangian: 
    #           L(u, beta, yc) = f(u) + beta/2*||A(u)-b||**2 + <yc, A(u) - b>
    #
    # INPUT:  myFunc:        Function handle to evaluate the function value and the gradient of augmented
    #                        lagrangian. Accepts 3 inputs: u, beta, yc
    #         f_grad_u:      Function handle to evaluate 
    #         proj:          Projection operator onto set C
    #         update_params: Function handle determining the update of
    #                        beta(penalty parameter) and sigma(dual step-size)
    #         gamma_rule:    Function handle determining the step size
    #         get_feas:      Function handle which returns (A(u) - b)
    #         params:        Structure containing parameters for the
    #                        algorithm( maxiters, Uinit, tol etc.)
    # OUTPUT: Ucur:       Last iterate of the algorithm
    #         output:     Structure containing the history of the
    #                     iterates(times, feas, obj, grad_mapping etc.) 

    # Initializing the various parameters and output structures
    tol = 1e-15;
    output = init_hist_track(params)
    Ucur, yc, ysum, Usum, gamma_, beta_, beta_0,sigma_ = init_iters(params)

    t = time.time(); start = t
    
    current_obj = float('Inf'); prev_obj = float('Inf')
    beta_old = beta_; sigma_old = sigma_; feas_old = float('Inf')
    Uold = Ucur
    
    tot_iter = 0
    m_s = 50
    omega_ = 1.001    
    # Starting the iteration
    for ii in range(1,params['iters']):
        Uold = Ucur
        m_s = m_s*omega_
        
        funcEval = lambda u: myFunc(u, yc, beta_)
        step_size = gamma_rule(1/beta_, yc)

        # Do one iteration of iALM algorithm - we use here accelerated proximal gradient descent.
        Ucur,_ ,nfeval, gamma_ = solve_apgm_clustering(funcEval,Uold,m_s,proj,step_size);
        
        # Update the output structures
        output = track_hist(ii-1, output, start, Ucur, Uold, beta_,sigma_,gamma_,yc,get_feas, funcEval,tot_iter,f_u)
        verbosity(ii-1, params, output,tot_iter)
        
        # Update the relevant parameters
        feas_cur = get_feas(Ucur)
        yc      = yc + sigma_ * feas_cur;
        
        beta_ = beta_*omega_**(1/4)
        sigma_ = min(params['sigma0']/sqrt(ii),1e2*params['sigma0']/(ii*np.linalg.norm(feas_cur,2)*np.log(ii+1)**2))
        
        # Update storing variables
        beta_old = beta_
        sigma_old = sigma_
        feas_old = np.linalg.norm(feas_cur,2)
        tot_iter = tot_iter + nfeval #Number of function evaluations performed
        Uav = Ucur
        ycav = yc

        # Stopping criteria
        if (abs(current_obj-prev_obj) < tol and feas_old < 1e-6):
            print('Converged: Breaking...')
            break;
        
        prev_obj = current_obj

        if tot_iter > params['iters']:
            print('Max iters reached: Breaking...\n')
            break;

    return Ucur, output, yc

Main optimization

In [ ]:
Udl, output_iAL, yc = apgm_al_clustering(myFunc, proj, gamma_rule, get_feas, params)

### Visualize the results

In [ ]:
plot_compare_result(output_CGM,output_iAL, X, Udl.dot(Udl.T))

### Get the assignments from the result of the SDP



In [ ]:
center_iAL,assign_iAL = sdp_rounding(Udl.dot(Udl.T),10)

In [ ]:
print('Misclassification rate for CGM: {:.4f}'.format(misclassification_rate(assign_CGM)))
print('Misclassification rate for iAL: {:.4f}'.format(misclassification_rate(assign_iAL)))

In [ ]:
vis_samples(assign_iAL)

### 3. Questions (II)
#### <span style="font-variant:small-caps;">Question 3 *(Interpretation)*</span> 
---
What convergence rate do you observe for CGM and iAL? Do you have any interpretation on the performances you observe?

What are the misclassification rates for both methods? Are they close? Why is this so?

---

#### <span style="font-variant:small-caps;">Question 4 *(Coding & Interpretation)*</span> 
---

Play around with the rank of the factorization, $r$, of $\mathbf{V}$. Set it to less than $10$, $10$, $15$ and run the iAL algorithm. What do you observe?

---


## References:
1. <span style="font-variant:small-caps;"> Peng, J., & Wei, Y. </span>(2007). Approximating k-means-type clustering via semidefinite programming. *SIAM journal on optimization*, 18(1), 186-205.
2. <span style="font-variant:small-caps;"> Yurtsever, A., Fercoq, O., Locatello, F., & Cevher, V. </span> (2018). A conditional gradient framework for composite convex minimization with applications to semidefinite programming. *arXiv preprint arXiv:1804.08544*.
3. <span style="font-variant:small-caps;"> Mixon, D. G., Villar, S., & Ward, R. </span> (2016). Clustering subgaussian mixtures by semidefinite programming. *arXiv preprint arXiv:1602.06612*.
4. <span style="font-variant:small-caps;"> Xiao, H., Rasul, K., & Vollgraf, R. </span> (2017). Fashion-mnist: a novel image dataset for benchmarking machine learning algorithms. *arXiv preprint arXiv:1708.07747*.
5. <span style="font-variant:small-caps;">Jaggi, M. </span> (2013, June). Revisiting Frank-Wolfe: Projection-Free Sparse Convex Optimization. In *ICML* (1) (pp. 427-435).
6. <span style="font-variant:small-caps;">Sahin, M. F., Eftekhari, A., Alacaoglu, A., Latorre, F., & Cevher, V.   </span> (2019) An Inexact Augmented Lagrangian Framework for Nonconvex Optimization with Nonlinear Constraints. *arXiv preprint arXiv:1906.11357*.
7. <span style="font-variant:small-caps;">Kulis, B., Surendran, A. C., & Platt, J. C.</span> (2007). Fast low-rank semidefinite programming for embedding and clustering. *Artificial Intelligence and Statistics*, pages 235–242. 